In [73]:
import sys
from difflib import get_close_matches
from Levenshtein import distance
import pandas as pd
import time

sp_strain = {}
gtdb_metadata_file = open("GTDB_QUALITY_REPRESENTATIVE_GENOMES_v202", "r")

gtdb_entry_taxomies_dict = {}
gtdb_entry_ncbi_tax_id_dict = {}
sp_strain = {}
counter = 0

for line in gtdb_metadata_file:
    if counter < 1:
        counter += 1
        continue
    line = line.split("\t")

    gtdb_accession_id = line[0]
    ncbi_genbank_accession_id = line[54]
    ncbi_tax_id = line[77]
    ncbi_species_id = line[73]

    gtdb_taxonomy = line[16]
    lsu_silva_taxonomy = line[37]
    ncbi_species_name = line[62]
    ncbi_taxonomy = line[78]
    ncbi_taxonomy_unfiltered = line[79]
    # ssu_gg_taxonomy = line[97]
    ssu_silva_taxonomy = line[105]

    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id] = {}
    gtdb_entry_ncbi_tax_id_dict[ncbi_genbank_accession_id] = ncbi_tax_id
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id] = []
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id].append(gtdb_taxonomy.split(";")[-1])
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id].append(lsu_silva_taxonomy.split(";")[-1])
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id].append(ncbi_species_name)
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id].append(ncbi_taxonomy.split(";")[-1])
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id].append(ncbi_taxonomy_unfiltered.split(";")[-1])
    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id].append(ssu_silva_taxonomy.split(";")[-1])

    gtdb_entry_taxomies_dict[ncbi_genbank_accession_id] = list(set(gtdb_entry_taxomies_dict[ncbi_genbank_accession_id]))


    if ncbi_species_id not in sp_strain:
        sp_strain[ncbi_species_id] = []
        sp_strain[ncbi_species_id].append(ncbi_tax_id)
    else:
        sp_strain[ncbi_species_id].append( ncbi_tax_id)



from matplotlib import pyplot as plt
from collections import Counter

g = {k:len(v) for k,v in sp_strain.items()}

f = list(sp_strain.keys())
h = [j for j in g.values()]

# plt.bar( range(len( f )), h  )



histogram = Counter(min(mark // len(h) * len(h), 90) for mark in h)
counter = 0 
for i,j in g.items():
    if j>40:
        counter += 1 
        print(i)
print(counter)




2026780
1883427
2030927
1898104
2026724
1913989
9
1898207
1898204
194843
172733
370804
1909294
165190
297314
165186
152509
77133
86027
28037
294
21


In [63]:
import numpy as np
# Create a dictionary with test scores
scores_dict = g

# Extract the scores from the dictionary
scores = list(scores_dict.values())

# Calculate the range of the data
data_range = max(scores) - min(scores)

# Calculate the number of bins
num_bins = int(round(data_range / 5))

# Create an array of evenly spaced bins
bins = np.arange(min(scores), max(scores) + 3, 3)

# Create the histogram
# plt.hist(scores, bins=bins, color='steelblue', edgecolor='black', linewidth=1.2)

[  1   4   7  10  13  16  19  22  25  28  31  34  37  40  43  46  49  52
  55  58  61  64  67  70  73  76  79  82  85  88  91  94  97 100 103 106
 109 112 115 118]


In [89]:

def diff(users_species_name, map_dict):

    # option B 
    lev_distances = {}
    best_lev_distance = 1000
    
    for genomeId, taxonomy_names in map_dict.items():
        for tax_name in taxonomy_names:
            lev_distance = distance(users_species_name, tax_name)
            if lev_distance < best_lev_distance:
                best_lev_distance = lev_distance
                best_hit_id = genomeId
                lev_distances[genomeId] = best_lev_distance


    if best_lev_distance > 0.9*len(users_species_name) :

        print(">>> high levenstein distance" )
        print(users_species_name)
        print(lev_distances)
        print("<<<<<<")


    return best_hit_id
    

genomeId = diff("Lachnospiraceae_FCS020_group bacterium", gtdb_entry_taxomies_dict)

print(genomeId)
print(gtdb_entry_ncbi_tax_id_dict[genomeId])


GCA_003862155.1
1898203


In [95]:

kostas_species = open("/home/luna.kuleuven.be/u0156635/Downloads/20230521_433_SANICSII_REVEAL_taxa.csv","r")
counter = 0 
for line in kostas_species:

    line = line[:-1]
    line = line.split(",")[-1]
    line = line.split(";")

    if counter < 1 or line[-1] == "NA":
        counter += 1 
        continue


    species = line[-2] + " " + line[-1]
    species = species.replace("NA","")


    if len(species) < 3:
        continue


    closest_genome = diff(species, gtdb_entry_taxomies_dict)
    closest_ncbi   = gtdb_entry_ncbi_tax_id_dict[closest_genome]


    print("kostas species:", species, "~~~", closest_genome, closest_ncbi)

    counter += 1
    if counter >1000:
        break




kostas species: Lactobacillus delbrueckii ~~~ GCA_001433875.1 1423823
kostas species: Lactobacillus fermentum/mucosae ~~~ GCA_000159215.1 525325
kostas species: Streptococcus salivarius/thermophilus ~~~ GCA_002549775.1 853
kostas species: Erysipelatoclostridium ramosum ~~~ GCA_014131695.1 445974
kostas species: Catenibacterium mitsuokai ~~~ GCA_000173795.1 451640
kostas species: Coprobacillus cateniformis ~~~ GCA_001312445.1 1297794
kostas species: Holdemanella biformis ~~~ GCA_000156655.1 518637
kostas species: Akkermansia muciniphila ~~~ GCA_002885095.1 239935
kostas species: Bacteroides uniformis ~~~ GCA_004793475.1 2516960
kostas species: Bacteroides intestinalis ~~~ GCA_007896885.1 2600309
kostas species: Bacteroides cellulosilyticus/intestinalis ~~~ GCA_000158035.1 537012
kostas species: Bacteroides faecis/thetaiotaomicron ~~~ GCA_900106755.1 1077285
kostas species: Prevotella_9 copri ~~~ GCA_002224675.1 165179
kostas species: Odoribacter splanchnicus ~~~ GCA_000190535.1 709991
k